In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import math
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

使用设备: cuda


In [103]:
class AdditionNet(nn.Module):
    def __init__(self):
        super(AdditionNet, self).__init__()
        # 简单的三层网络
        self.fc1 = nn.Linear(6, 256)  # 输入层
        self.fc2 = nn.Linear(256, 1024) # 隐藏层
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 256)
        self.fc5 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.softmax(self.fc5(x), dim=1)  # 使用softmax确保两个输出和为1
        return x

In [143]:
def normalize_data_log(a, b, c, max_value=10000):
    """将数值分解为整数部分和小数部分，根据数量级设置不同精度"""
    # 确定小数保留位数
    if max_value <= 10:
        decimal_places = 1
    elif max_value <= 100:
        decimal_places = 2
    elif max_value <= 1000:
        decimal_places = 3
    else:
        decimal_places = 4
    
    # 截断到指定小数位
    factor = 10**decimal_places
    a_truncated = math.floor(a * factor) / factor
    b_truncated = math.floor(b * factor) / factor
    c_truncated = math.floor(c * factor) / factor
    
    # 分离整数和小数部分
    a_int, a_dec = int(a_truncated), a_truncated - int(a_truncated)
    b_int, b_dec = int(b_truncated), b_truncated - int(b_truncated)
    c_int, c_dec = int(c_truncated), c_truncated - int(c_truncated)
    
    # 使用log1p归一化整数部分
    max_log = np.log1p(max_value * 2)
    a_int_norm = np.log1p(a_int) / max_log
    b_int_norm = np.log1p(b_int) / max_log
    c_int_norm = np.log1p(c_int) / max_log
    
    # 小数部分只保留指定精度的位数，并放大
    decimal_scale = 10**decimal_places
    a_dec_scaled = int(a_dec * decimal_scale)
    print(a_dec_scaled)
    b_dec_scaled = int(b_dec * decimal_scale)
    c_dec_scaled = int(c_dec * decimal_scale)
    
    # 再使用log1p归一化小数部分
    max_dec_log = np.log1p(max_value * 2)
    a_dec_norm = np.log1p(a_dec_scaled) / max_dec_log
    b_dec_norm = np.log1p(b_dec_scaled) / max_dec_log
    c_dec_norm = np.log1p(c_dec_scaled) / max_dec_log
    
    return [a_int_norm, a_dec_norm, b_int_norm, b_dec_norm, c_int_norm, c_dec_norm]

In [114]:
def generate_mixed_data(num_samples=80000):
    """生成混合难度的数据集，包含不同范围的加法问题和困难案例"""
    data = []
    labels = []
    
    # 每个难度范围的样本数量
    samples_per_range = num_samples // 4  # 平均分配到4个难度级别
    
    # 生成四个不同难度的数据
    for max_value in [10, 100, 1000, 10000]:
        # 普通样本数量
        base_samples = int(samples_per_range * 0.7)
        hard_samples = samples_per_range - base_samples
        
        # 普通样本
        for _ in range(base_samples):
            a = np.random.uniform(0, max_value)
            b = np.random.uniform(0, max_value)
            
            if np.random.random() < 0.5:
                c = a + b  # 正确答案
                label = [0, 1]
            else:
                error_rate = np.random.uniform(0.01, 0.1)
                error = (a + b) * error_rate * (1 if np.random.random() < 0.5 else -1)
                c = a + b + error
                label = [1, 0]
            
            features = normalize_data_log(a, b, c, 10000)
            data.append(features)
            labels.append(label)
        
        # 困难样本：一大一小
        n_hard1 = hard_samples // 3
        for _ in range(n_hard1):
            a = np.random.uniform(max_value*0.8, max_value)
            b = np.random.uniform(0.0001, 0.01)
            
            if np.random.random() < 0.5:
                c = a + b
                label = [0, 1]
            else:
                error = np.random.uniform(0.0001, 0.001) * (1 if np.random.random() < 0.5 else -1)
                c = a + b + error
                label = [1, 0]
            
            features = normalize_data_log(a, b, c, 10000)
            data.append(features)
            labels.append(label)
        
        # 困难样本：小数位精度测试
        n_hard2 = hard_samples - n_hard1
        for _ in range(n_hard2):
            a = np.random.uniform(0, max_value)
            b = np.random.uniform(0, max_value)
            correct_sum = a + b
            
            if np.random.random() < 0.5:
                c = correct_sum
                label = [0, 1]
            else:
                # 非常接近但不等于正确答案
                tiny_error = np.random.uniform(0.00001, 0.0001) * (1 if np.random.random() < 0.5 else -1)
                c = correct_sum + tiny_error
                label = [1, 0]
                
            features = normalize_data_log(a, b, c, 10000)
            data.append(features)
            labels.append(label)
    
    # 打乱数据
    combined = list(zip(data, labels))
    np.random.shuffle(combined)
    data, labels = zip(*combined)
    
    return torch.tensor(data, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)



In [115]:
def check_addition(model, a, b, c, max_value=10000):
    model.eval()
    with torch.no_grad():
        # 获取适当的精度
        if max_value <= 10:
            decimal_places = 1
        elif max_value <= 100:
            decimal_places = 2
        elif max_value <= 1000:
            decimal_places = 3
        else:
            decimal_places = 4
            
        # 将a, b, c截断到指定小数位
        factor = 10**decimal_places
        a_truncated = math.floor(a * factor) / factor
        b_truncated = math.floor(b * factor) / factor
        c_truncated = math.floor(c * factor) / factor
        
        # 验证加法是否正确，只考虑到指定小数位
        correct = abs((a_truncated + b_truncated) - c_truncated) < (0.5 * 10**(-decimal_places))
        
        # 使用模型预测
        features = normalize_data_log(a, b, c, max_value)
        input_data = torch.tensor([features], dtype=torch.float32).to(device)
        probabilities = model(input_data)
        
        return probabilities[0, 1].item(), correct

In [116]:
def curriculum_training(model, loss_fn, optimizer, epochs_per_stage=150, final_stage_epochs=300, stage_losss_break=0.5, final_accuracy_break=0.95):
    stages = [
        {"max_value": 10, "name": "10以内加法", "decimal_precision": 1},
        {"max_value": 100, "name": "100以内加法", "decimal_precision": 2},
        {"max_value": 1000, "name": "1000以内加法", "decimal_precision": 3},
        {"max_value": 10000, "name": "10000以内加法", "decimal_precision": 4}
    ]
    
    # 为每个阶段保存一个检查点
    for stage_idx, stage in enumerate(stages):
        print(f"\n开始训练阶段 {stage_idx+1}: {stage['name']}")
        max_value = stage["max_value"]
        decimal_precision = stage["decimal_precision"]
        
        # 为当前难度生成数据
        X_train, y_train = generate_data(num_samples=8000, max_value=max_value)
        X_val, y_val = generate_data(num_samples=1000, max_value=max_value)
        
        # 将数据移到设备
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        X_val = X_val.to(device)
        y_val = y_val.to(device)
        
        # 训练当前阶段
        batch_size = 128
        for epoch in range(epochs_per_stage):
            # 训练模式
            model.train()
            total_loss = 0
            indices = torch.randperm(len(X_train))
            
            for i in range(0, len(X_train), batch_size):
                batch_indices = indices[i:i+batch_size]
                inputs = X_train[batch_indices]
                targets = y_train[batch_indices]
                
                # 前向传播
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                
                # 反向传播和优化
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            train_loss = total_loss / (len(X_train) / batch_size)
            
            # 验证
            model.eval()
            with torch.no_grad():
                val_outputs = model(X_val)
                val_loss = loss_fn(val_outputs, y_val).item()
                
                # 计算准确率 - 使用当前阶段的小数精度要求
                val_accuracy = evaluate_with_precision(model, X_val, y_val, decimal_precision)
            
            print(f'阶段 {stage_idx+1} 轮次 {epoch+1}/{epochs_per_stage}, '
                  f'训练损失: {train_loss:.4f}, 验证损失: {val_loss:.4f}, '
                  f'验证准确率: {val_accuracy:.4f}')
            
            # 如果验证准确率达到高水平，提前进入下一阶段
            if val_loss < stage_losss_break:
                print(f"阶段 {stage_idx+1} 提前完成! 验证损失: {val_loss:.4f}")
                break
                
        # 保存阶段检查点
        torch.save(model.state_dict(), f"model_stage_{stage_idx+1}.pt")
        
    # 最终阶段：混合数据训练
    print("\n开始最终整合阶段: 混合数据训练")
    X_train, y_train = generate_mixed_data()  # 生成各个难度级别的混合数据
    X_val, y_val = generate_mixed_data(num_samples=2000)
    
    # 将数据移到设备
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_val = X_val.to(device)
    y_val = y_val.to(device)
    batch_size = 1024
    
    # 最终训练
    for epoch in range(final_stage_epochs):
        # 训练模式
        model.train()
        total_loss = 0
        indices = torch.randperm(len(X_train))
        
        for i in range(0, len(X_train), batch_size):
            batch_indices = indices[i:i+batch_size]
            inputs = X_train[batch_indices]
            targets = y_train[batch_indices]
            
            # 前向传播
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        train_loss = total_loss / (len(X_train) / batch_size)
        
        # 验证 - 使用最严格的精度要求
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = loss_fn(val_outputs, y_val).item()
            
            # 使用最高精度要求评估
            val_accuracy = evaluate_with_precision(model, X_val, y_val, decimal_precision=4)
        
        print(f'最终阶段 轮次 {epoch+1}/{final_stage_epochs}, '
              f'训练损失: {train_loss:.4f}, 验证损失: {val_loss:.4f}, '
              f'验证准确率: {val_accuracy:.4f}')
        if val_accuracy > final_accuracy_break:
            print(f"最终阶段 提前完成! 验证准确率: {val_accuracy:.4f}")
            break
    
    # 保存最终模型
    torch.save(model.state_dict(), "model_final.pt")
    return model

def evaluate_with_precision(model, X, y, decimal_precision):
    """根据指定的小数精度评估模型准确率"""
    outputs = model(X)
    _, predicted = torch.max(outputs, 1)
    _, true_labels = torch.max(y, 1)
    
    # 使用原始数据重建a,b,c
    # 注意：这里需要修改，因为数据格式改变了，需要从特征中提取回a,b,c
    # 假设这里有一个函数可以从特征中恢复原始值
    
    correct_count = 0
    total_count = len(X)
    
    for i in range(total_count):
        # 获取预测标签和真实标签
        pred_label = predicted[i].item()
        true_label = true_labels[i].item()
        
        # 预测标签和真实标签一致，直接计数为正确
        if pred_label == true_label:
            correct_count += 1
        
    return correct_count / total_count

In [117]:
# X_train, y_train = generate_data(num_samples=80000, max_value=10000)
# X_val, y_val = generate_data(num_samples=1000, max_value=10000)
# X_test, y_test = generate_data(num_samples=2000, max_value=10000)

# # 将数据移到GPU
# X_train = X_train.to(device)
# y_train = y_train.to(device)
# X_val = X_val.to(device)
# y_val = y_val.to(device)
# X_test = X_test.to(device)
# y_test = y_test.to(device)

def init_xavier(m):
    if isinstance(m, nn.Linear):
        # 使用Xavier均匀分布初始化权重
        nn.init.xavier_uniform_(m.weight)
        # 将偏置初始化为小的正数，避免ReLU神经元"死亡"
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.01)

# 创建模型并移到GPU
model = AdditionNet().to(device)
model.apply(init_xavier)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model = curriculum_training(model, criterion, optimizer, epochs_per_stage=150, stage_losss_break=0.3, 
                            final_accuracy_break=0.96)

# 训练模型
# epochs = 300
# batch_size = 1024
# losses = []
# best_val_loss = float('inf')
# patience = 100  # 早停耐心值
# no_improve = 0
# accuracy_threshold = 0.996

# for epoch in range(epochs):
# 	model.train()
# 	total_loss = 0
# 	# 获取随机批次
# 	indices = torch.randperm(len(X_train))
# 	for i in range(0, len(X_train), batch_size):
# 		batch_indices = indices[i:i+batch_size]
# 		inputs = X_train[batch_indices]
# 		targets = y_train[batch_indices]
		
# 		# 前向传播
# 		outputs = model(inputs)
# 		loss = criterion(outputs, targets)
		
# 		# 反向传播和优化
# 		optimizer.zero_grad()
# 		loss.backward()
# 		optimizer.step()
		
# 		total_loss += loss.item()
		
# 	train_loss = total_loss / (len(X_train) / batch_size)
# 	model.eval()
# 	with torch.no_grad():
# 		val_outputs = model(X_val)
# 		val_loss = criterion(val_outputs, y_val).item()
		
# 		_, val_predicted = torch.max(val_outputs, 1)
# 		_, val_true_labels = torch.max(y_val, 1)
# 		val_accuracy = (val_predicted == val_true_labels).float().mean().item()

# 	print(f'轮次 {epoch+1}/{epochs}, 训练损失: {train_loss:.4f}, 验证损失: {val_loss:.4f}, 验证准确率: {val_accuracy:.4f}')

# 	# 早停检查
# 	if val_loss < best_val_loss:
# 		best_val_loss = val_loss
# 		torch.save(model.state_dict(), 'best_model.pt')
# 		no_improve = 0
# 	else:
# 		no_improve += 1

# 	# 准确率达到阈值时停止
# 	if val_accuracy >= accuracy_threshold:
# 		print(f'验证准确率达到 {val_accuracy:.4f}，在第 {epoch+1} 轮停止训练')
# 		break
		
# 	# 早停检查
# 	if no_improve >= patience:
# 		print(f'连续 {patience} 轮未改善，在第 {epoch+1} 轮停止训练')
# 		break

# # 修改check_addition函数以使用GPU
# def check_addition(a, b, c, max_value=10000):
#     model.eval()
#     with torch.no_grad():
#         a_norm, b_norm, c_norm = normalize_data_log(a, b, c, max_value)
#         input_data = torch.tensor([[a_norm, b_norm, c_norm]], dtype=torch.float32).to(device)
#         probabilities = model(input_data)
#         # 打印原始输出，帮助调试
#         print(f"原始输出: {probabilities.cpu().numpy()}")
#         return probabilities[0, 1].item()  # 返回正确的概率(索引1)

# 测试示例
decimal_examples = [
    # 1-10：简单小数加法（正确结果）
    (12.34, 56.78, 69.12),       # 正确
    (123.45, 678.90, 802.35),    # 正确
    (0.75, 0.25, 1.00),          # 正确
    (9.99, 1.01, 11.00),         # 正确
    (45.67, 32.33, 78.00),       # 正确
    
    # 11-20：简单小数加法（错误结果）
    (22.22, 33.33, 56.66),       # 错误（正确应为55.55）
    (88.88, 11.11, 101.00),      # 错误（正确应为99.99）
    (7.50, 2.50, 9.50),          # 错误（正确应为10.00）
    (45.50, 45.50, 90.00),       # 错误（正确应为91.00）
    (67.89, 12.10, 81.99),       # 错误（正确应为79.99）
    
    # 21-30：大数加法（正确结果）
    (1234.56, 7890.12, 9124.68), # 正确
    (8765.43, 1234.57, 10000.00),# 正确
    (5432.10, 5432.10, 10864.20),# 正确
    (9999.99, 0.01, 10000.00),   # 正确
    (4567.89, 3210.11, 7778.00), # 正确
    
    # 31-40：大数加法（错误结果）
    (2345.67, 8901.23, 11000.00),# 错误（正确应为11246.90）
    (7777.77, 2222.22, 9999.00), # 错误（正确应为9999.99）
    (4444.44, 5555.55, 9999.00), # 错误（正确应为9999.99）
    (6789.01, 2345.67, 9035.68), # 错误（正确应为9134.68）
    (5000.00, 5000.00, 9000.00), # 错误（正确应为10000.00）
    
    # 41-50：更复杂的小数加法（正确结果）
    (123.456, 876.544, 1000.000),# 正确
    (0.1234, 0.8766, 1.0000),    # 正确
    (7.5432, 2.4568, 10.0000),   # 正确
    (99.9999, 0.0001, 100.0000), # 正确
    (567.123, 432.877, 1000.000),# 正确
    
    # 51-60：更复杂的小数加法（错误结果）
    (456.789, 543.211, 999.999), # 错误（正确应为1000.000）
    (1.2345, 2.3456, 3.5000),    # 错误（正确应为3.5801）
    (6.7890, 3.2109, 9.9998),    # 错误（正确应为9.9999）
    (0.0123, 0.0456, 0.0580),    # 错误（正确应为0.0579）
    (7.1234, 8.9876, 16.0000),   # 错误（正确应为16.1110）
    
    # 61-70：接近相等的数加法（正确结果）
    (999.999, 999.999, 1999.998),# 正确
    (555.555, 555.555, 1111.110),# 正确
    (123.123, 123.123, 246.246), # 正确
    (7.7777, 7.7777, 15.5554),   # 正确
    (0.5000, 0.5000, 1.0000),    # 正确
    
    # 71-80：接近相等的数加法（错误结果）
    (444.444, 444.444, 888.000), # 错误（正确应为888.888）
    (777.777, 777.777, 1555.555),# 错误（正确应为1555.554）
    (3.3333, 3.3333, 6.6000),    # 错误（正确应为6.6666）
    (1.1111, 1.1111, 2.2000),    # 错误（正确应为2.2222）
    (9.9999, 9.9999, 19.0000),   # 错误（正确应为19.9998）
    
    # 81-90：一个数很大一个数很小加法（正确结果）
    (9999.99, 0.01, 10000.00),   # 正确
    (0.0001, 999.9999, 1000.0000),# 正确
    (0.1234, 987.8766, 988.0000),# 正确
    (1.0000, 9999.0000, 10000.0000),# 正确
    (0.0001, 0.9999, 1.0000),    # 正确
    
    # 91-100：一个数很大一个数很小加法（错误结果）
    (9876.54, 0.01, 9876.54),    # 错误（正确应为9876.55）
    (0.0001, 100.0000, 100.0002),# 错误（正确应为100.0001）
    (1234.56, 0.001, 1234.56),   # 错误（正确应为1234.561）
    (0.0001, 9999.9998, 9999.0000),# 错误（正确应为9999.9999）
    (0.1111, 9999.8888, 9999.9000)# 错误（正确应为9999.9999）
]

for a, b, c in decimal_examples:
    prob = check_addition(model, a, b, c)
    correct = abs((a + b) - c) < 1e-5
    print(f'{a:.4f} + {b:.4f} = {c:.4f} 的正确概率: {prob:.4f}, 实际上是: {"正确" if correct else "错误"}, 检验结果是否正确: {"正确" if (correct and prob > 0.5) or (not correct and prob < 0.5) else "错误"}')
# 统计检验结果与实际结果相符的次数与百分比
total_examples = len(decimal_examples)
correct_predictions = 0

for a, b, c in decimal_examples:
    prob = check_addition(model,a, b, c)
    correct = abs((a + b) - c) < 1e-5
    prediction_correct = (correct and prob > 0.5) or (not correct and prob < 0.5)
    
    if prediction_correct:
        correct_predictions += 1

accuracy = correct_predictions / total_examples * 100
print(f"\n检验结果统计:")
print(f"总样本数: {total_examples}")
print(f"检验结果与实际结果相符的次数: {correct_predictions}")
print(f"检验准确率: {accuracy:.2f}%")
    


开始训练阶段 1: 10以内加法
阶段 1 轮次 1/150, 训练损失: 0.6979, 验证损失: 0.6923, 验证准确率: 0.5140
阶段 1 轮次 2/150, 训练损失: 0.6928, 验证损失: 0.6860, 验证准确率: 0.5360
阶段 1 轮次 3/150, 训练损失: 0.6851, 验证损失: 0.6769, 验证准确率: 0.5800
阶段 1 轮次 4/150, 训练损失: 0.6620, 验证损失: 0.6772, 验证准确率: 0.5680
阶段 1 轮次 5/150, 训练损失: 0.5938, 验证损失: 0.5837, 验证准确率: 0.6610
阶段 1 轮次 6/150, 训练损失: 0.5705, 验证损失: 0.5527, 验证准确率: 0.6830
阶段 1 轮次 7/150, 训练损失: 0.5416, 验证损失: 0.5540, 验证准确率: 0.6900
阶段 1 轮次 8/150, 训练损失: 0.5260, 验证损失: 0.5343, 验证准确率: 0.6870
阶段 1 轮次 9/150, 训练损失: 0.5078, 验证损失: 0.5166, 验证准确率: 0.6950
阶段 1 轮次 10/150, 训练损失: 0.5089, 验证损失: 0.5221, 验证准确率: 0.6970
阶段 1 轮次 11/150, 训练损失: 0.5128, 验证损失: 0.5187, 验证准确率: 0.7040
阶段 1 轮次 12/150, 训练损失: 0.4891, 验证损失: 0.5133, 验证准确率: 0.7060
阶段 1 轮次 13/150, 训练损失: 0.5037, 验证损失: 0.5259, 验证准确率: 0.6880
阶段 1 轮次 14/150, 训练损失: 0.4933, 验证损失: 0.5073, 验证准确率: 0.7280
阶段 1 轮次 15/150, 训练损失: 0.4658, 验证损失: 0.4917, 验证准确率: 0.7250
阶段 1 轮次 16/150, 训练损失: 0.4502, 验证损失: 0.4889, 验证准确率: 0.7280
阶段 1 轮次 17/150, 训练损失: 0.4482, 验证损失: 0.5227, 验证准确率: 0.7110
阶段 1 

KeyboardInterrupt: 

In [ ]:
model = AdditionNet().to(device)
model.load_state_dict(torch.load("model_final.pt"))
decimal_examples = [
    # 1-10：简单小数加法（正确结果）
    (12.34, 56.78, 69.12),       # 正确
    (123.45, 678.90, 802.35),    # 正确
    (0.75, 0.25, 1.00),          # 正确
    (9.99, 1.01, 11.00),         # 正确
    (45.67, 32.33, 78.00),       # 正确
    
    # 11-20：简单小数加法（错误结果）
    (22.22, 33.33, 56.66),       # 错误（正确应为55.55）
    (88.88, 11.11, 101.00),      # 错误（正确应为99.99）
    (7.50, 2.50, 9.50),          # 错误（正确应为10.00）
    (45.50, 45.50, 90.00),       # 错误（正确应为91.00）
    (67.89, 12.10, 81.99),       # 错误（正确应为79.99）
    
    # 21-30：大数加法（正确结果）
    (1234.56, 7890.12, 9124.68), # 正确
    (8765.43, 1234.57, 10000.00),# 正确
    (5432.10, 5432.10, 10864.20),# 正确
    (9999.99, 0.01, 10000.00),   # 正确
    (4567.89, 3210.11, 7778.00), # 正确
    
    # 31-40：大数加法（错误结果）
    (2345.67, 8901.23, 11000.00),# 错误（正确应为11246.90）
    (7777.77, 2222.22, 9999.00), # 错误（正确应为9999.99）
    (4444.44, 5555.55, 9999.00), # 错误（正确应为9999.99）
    (6789.01, 2345.67, 9035.68), # 错误（正确应为9134.68）
    (5000.00, 5000.00, 9000.00), # 错误（正确应为10000.00）
    
    # 41-50：更复杂的小数加法（正确结果）
    (123.456, 876.544, 1000.000),# 正确
    (0.1234, 0.8766, 1.0000),    # 正确
    (7.5432, 2.4568, 10.0000),   # 正确
    (99.9999, 0.0001, 100.0000), # 正确
    (567.123, 432.877, 1000.000),# 正确
    
    # 51-60：更复杂的小数加法（错误结果）
    (456.789, 543.211, 999.999), # 错误（正确应为1000.000）
    (1.2345, 2.3456, 3.5000),    # 错误（正确应为3.5801）
    (6.7890, 3.2109, 9.9998),    # 错误（正确应为9.9999）
    (0.0123, 0.0456, 0.0580),    # 错误（正确应为0.0579）
    (7.1234, 8.9876, 16.0000),   # 错误（正确应为16.1110）
    
    # 61-70：接近相等的数加法（正确结果）
    (999.999, 999.999, 1999.998),# 正确
    (555.555, 555.555, 1111.110),# 正确
    (123.123, 123.123, 246.246), # 正确
    (7.7777, 7.7777, 15.5554),   # 正确
    (0.5000, 0.5000, 1.0000),    # 正确
    
    # 71-80：接近相等的数加法（错误结果）
    (444.444, 444.444, 888.000), # 错误（正确应为888.888）
    (777.777, 777.777, 1555.555),# 错误（正确应为1555.554）
    (3.3333, 3.3333, 6.6000),    # 错误（正确应为6.6666）
    (1.1111, 1.1111, 2.2000),    # 错误（正确应为2.2222）
    (9.9999, 9.9999, 19.0000),   # 错误（正确应为19.9998）
    
    # 81-90：一个数很大一个数很小加法（正确结果）
    (9999.99, 0.01, 10000.00),   # 正确
    (0.0001, 999.9999, 1000.0000),# 正确
    (0.1234, 987.8766, 988.0000),# 正确
    (1.0000, 9999.0000, 10000.0000),# 正确
    (0.0001, 0.9999, 1.0000),    # 正确
    
    # 91-100：一个数很大一个数很小加法（错误结果）
    (9876.54, 0.01, 9876.54),    # 错误（正确应为9876.55）
    (0.0001, 100.0000, 100.0002),# 错误（正确应为100.0001）
    (1234.56, 0.001, 1234.56),   # 错误（正确应为1234.561）
    (0.0001, 9999.9998, 9999.0000),# 错误（正确应为9999.9999）
    (0.1111, 9999.8888, 9999.9000)# 错误（正确应为9999.9999）
]

for a, b, c in decimal_examples:
    prob = check_addition(a, b, c)
    correct = abs((a + b) - c) < 1e-5
    print(f'{a:.4f} + {b:.4f} = {c:.4f} 的正确概率: {prob:.4f}, 实际上是: {"正确" if correct else "错误"}, 检验结果是否正确: {"正确" if (correct and prob > 0.5) or (not correct and prob < 0.5) else "错误"}')
# 统计检验结果与实际结果相符的次数与百分比
total_examples = len(decimal_examples)
correct_predictions = 0

for a, b, c in decimal_examples:
    prob = check_addition(a, b, c)
    correct = abs((a + b) - c) < 1e-5
    prediction_correct = (correct and prob > 0.5) or (not correct and prob < 0.5)
    
    if prediction_correct:
        correct_predictions += 1

accuracy = correct_predictions / total_examples * 100
print(f"\n检验结果统计:")
print(f"总样本数: {total_examples}")
print(f"检验结果与实际结果相符的次数: {correct_predictions}")
print(f"检验准确率: {accuracy:.2f}%")



原始输出: [[0.30360958 0.69639045]]
12.3400 + 56.7800 = 69.1200 的正确概率: 0.6964, 实际上是: 正确, 检验结果是否正确: 正确
原始输出: [[0.06405322 0.93594676]]
123.4500 + 678.9000 = 802.3500 的正确概率: 0.9359, 实际上是: 正确, 检验结果是否正确: 正确
原始输出: [[0.32635722 0.67364275]]
0.7500 + 0.2500 = 1.0000 的正确概率: 0.6736, 实际上是: 正确, 检验结果是否正确: 正确
原始输出: [[0.07651028 0.92348975]]
9.9900 + 1.0100 = 11.0000 的正确概率: 0.9235, 实际上是: 正确, 检验结果是否正确: 正确
原始输出: [[0.16146506 0.83853495]]
45.6700 + 32.3300 = 78.0000 的正确概率: 0.8385, 实际上是: 正确, 检验结果是否正确: 正确
原始输出: [[0.90496624 0.09503371]]
22.2200 + 33.3300 = 56.6600 的正确概率: 0.0950, 实际上是: 错误, 检验结果是否正确: 正确
原始输出: [[0.1937114 0.8062886]]
88.8800 + 11.1100 = 101.0000 的正确概率: 0.8063, 实际上是: 错误, 检验结果是否正确: 错误
原始输出: [[0.940557   0.05944296]]
7.5000 + 2.5000 = 9.5000 的正确概率: 0.0594, 实际上是: 错误, 检验结果是否正确: 正确
原始输出: [[0.57528    0.42472002]]
45.5000 + 45.5000 = 90.0000 的正确概率: 0.4247, 实际上是: 错误, 检验结果是否正确: 正确
原始输出: [[0.80311286 0.19688709]]
67.8900 + 12.1000 = 81.9900 的正确概率: 0.1969, 实际上是: 错误, 检验结果是否正确: 正确
原始输出: [[0.07178336 0.92821

In [146]:
normalize_data_log(1.111,2.2,3.3,max_value=10)

1


[np.float64(0.227670248696953),
 np.float64(0.2890648263178878),
 np.float64(0.3608488067145301),
 np.float64(0.45815690999132613),
 np.float64(0.455340497393906),
 np.float64(0.45815690999132613)]

In [37]:
abs((1234.56+0.001)-1234.56) < 1e-5


False